In [1]:
import sys
sys.path.append("../")

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.autograd.profiler as profiler

import apex.fp16_utils as fp16

import os
import numpy as np
from sklearn.datasets import make_classification
from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser

from utils.moduleCodeProfiler import rankByCriteria


In [2]:
!nvidia-smi

Sun Nov 22 17:06:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 455.32.00    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 000047DD:00:00.0 Off |                    0 |
| N/A   51C    P8    51W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
cuda0 = torch.device('cuda:0') 

In [4]:
parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter)
args = parser.parse_args('')

args.data_dir = '~/datadrive'
args.dataset_dir = 'toy_mlp_1'
args.seed = 123
args.batch_size = 1000
# https://stackoverflow.com/questions/15753701/how-can-i-pass-a-list-as-a-command-line-argument-with-argparse
args.hidden_layer_dims = [10, 10, 10, 10]
args.lr = 0.01
args.epochs = 20

# Toy Data Generation

In [5]:
# construct and save toydataset

m_train = 9000
m_total = m_train

X, y = make_classification(n_samples=m_total, n_features=10, n_informative=10, n_redundant=0, n_repeated=0, n_classes=5, n_clusters_per_class=2, weights=None, flip_y=0.01, class_sep=1.0, hypercube=True, shift=0.0, scale=1.0, shuffle=True, random_state=args.seed)
# y = np.expand_dims(y, -1)

np.random.seed(args.seed)
permutation = np.random.permutation(m_total)
print('First 10 training indices', permutation[:10])
print('X shape', X.shape)
print('y shape', y.shape)

train_indices = permutation[0:m_train]

dataset_dir = 'toy_mlp_1'
os.makedirs(os.path.join(args.data_dir, dataset_dir, 'train'), mode = 0o777, exist_ok = True) 

np.save(os.path.join(args.data_dir, dataset_dir, 'train', 'features.npy'), X[train_indices])
np.save(os.path.join(args.data_dir, dataset_dir, 'train', 'labels.npy'), y[train_indices])

First 10 training indices [1603 8472 2213  498 1038 8399 3324 7535 1519 1959]
X shape (9000, 10)
y shape (9000,)


In [6]:
class ToyDataset(Dataset):
    """Toy dataset construction."""

    def __init__(self, data_dir):
        """
        Args:
            data_dir (string): Path to the directory with data files.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        # shape (m, nx)
        self.X = np.load(os.path.join(data_dir, 'features.npy'))
        # shape (m, ny=1)
        self.y = np.load(os.path.join(data_dir, 'labels.npy'))
        

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        else:
            X = torch.from_numpy(self.X[idx, :]).type(torch.HalfTensor)
            y = torch.tensor(self.y[idx], dtype=torch.long)
#             y = torch.from_numpy(self.y[idx, :]).type(torch.FloatTensor)
            sample = {'X': X, 'y': y}

        return sample

# Model

In [7]:
class MLPLazy(nn.Module):

    def __init__(self, nx, hidden_layer_dims, ny):
        super(MLPLazy, self).__init__()
        self.hidden_layer_dims = hidden_layer_dims
        
        linear_layers = []
        last_dim = nx
        for next_dim in hidden_layer_dims:
            linear_layer = nn.Linear(last_dim, next_dim)
            linear_layers.append(linear_layer)
            last_dim = next_dim
        # should push to ModuleList so that params stay on cuda
        self.linear_layers = nn.ModuleList(linear_layers)
        self.scorer = nn.Linear(last_dim, ny)

    def forward(self, X):
        '''
        X has shape (m, nx)
        '''
        last_X = X
        for i, linear_layer in enumerate(self.linear_layers):
            # shape (m, self.hidden_layer_dims[i])
            last_X = linear_layer(last_X)
            # shape (m, self.hidden_layer_dims[i])
            last_X = torch.relu(last_X)
        # shape (m, ny)
        z = self.scorer(last_X)
        # shape (m, ny)
        a = torch.softmax(z, dim=1)
        return z, a

# Workflow

In [8]:
def check_weights_precision(model):
    '''check weight precisions for each layer of MLP'''
    for i, layer in enumerate(model.linear_layers):
        print(f'layer {i}, weight dtype {layer.weight.dtype}')
        print(f'layer {i}, bias dtype {layer.bias.dtype}')
    print(f'scorer weight dtype {model.scorer.weight.dtype}')
    print(f'scorer bias dtype {model.scorer.bias.dtype}')

In [9]:
def get_master(opt):
    '''create a float32 master copy of float16 model weights in optimizer'''
    model_pgs = [[param for param in pg['params'] if param.requires_grad] for pg in opt.param_groups]
    master_pgs = [[param.clone().float().detach() for param in pg] for pg in model_pgs]
    for pg in master_pgs:
        for param in pg: param.requires_grad_(True)
    return model_pgs, master_pgs

In [10]:
def push_master_to_optimizer(opt, master_pgs):
    '''
        link master copy pgs to optimizer, 
        keeping other hparams such as lr, momentum dampening, weight_decay...'''
    for opt_pg, master_pg in zip(opt.param_groups, master_pgs):
        opt_pg['params'] = master_pg

In [11]:
def to_master_grads(model_pgs, master_pgs, flat_master:bool=False):
    '''copy float16 gradients from model to float32 gradients in master copy of weights'''
    for (model_params,master_params) in zip(model_pgs,master_pgs):
        fp16.model_grads_to_master_grads(model_params, master_params, flat_master=flat_master)

In [12]:
def to_model_params(model_pgs, master_pgs, flat_master:bool=False)->None:
    '''copy master copy of updated weights in float32 to model weights in float 16'''
    for (model_params,master_params) in zip(model_pgs,master_pgs):
        fp16.master_params_to_model_params(model_params, master_params, flat_master=flat_master)

In [13]:
def scale_down_master_grad(master_pgs, loss_scale):
    '''
    scale down all gradients for all master param groups
    '''
    for master_params in master_pgs:
        for param in master_params:
            if param.grad is not None:
                param.grad.div_(loss_scale)

In [14]:
def get_max_memory_alloc():
    '''read and reset max memory allocation'''
    devices_max_memory_alloc = {}
    for i in range(torch.cuda.device_count()):
        device = torch.device(f'cuda:{i}')
        devices_max_memory_alloc[device] = torch.cuda.max_memory_allocated(device) / 1e6
        torch.cuda.reset_max_memory_allocated(device)
    return devices_max_memory_alloc

In [15]:
def main_train(args, gpu=0, debug=False):

    torch.manual_seed(args.seed)
    
    ################################################################
    # load datasets
    training_set = ToyDataset(data_dir=os.path.join(args.data_dir, args.dataset_dir, 'train'))
    training_generator = torch.utils.data.DataLoader(dataset=training_set, 
                                                        batch_size=args.batch_size, 
                                                        shuffle=True, 
                                                        num_workers=0, 
                                                        pin_memory=True)

    nx = training_set.X.shape[1]
    ny = max(training_set.y) + 1
    ################################################################
    loss_scale = 512

    # 1. Create model
    model = MLPLazy(nx, args.hidden_layer_dims, ny)  # single
    loss_criterion = nn.CrossEntropyLoss(reduction='mean')
    torch.cuda.set_device(gpu)
    model.to(device=gpu)    

    # 2. initialize optimizer
    opt = torch.optim.SGD(model.parameters(), lr=args.lr)  # half
    if debug:
        print('\nmodel weights at init')
        check_weights_precision(model)

    # 3. Cast model to float16
    fp16.convert_network(model, torch.float16)
    if debug:
        print('\nmodel weights after casting')
        check_weights_precision(model)

    # 4. Create a copy of this float16 model's weight in float32 as the master copy
    model_pgs, master_pgs = get_master(opt)  # half, single

    # 5. replace optimizer float16 weights with float32 master copy
    push_master_to_optimizer(opt, master_pgs)  # opt single

    def check_grad():
        print('optimizer grad:\n', opt.param_groups[0]['params'][0].grad)
        print('master pg grad:\n', master_pgs[0][0].grad)
        print('model pg grad:\n', model_pgs[0][0].grad)

    def check_weights():
        print('optimizer weights:\n', opt.param_groups[0]['params'][0])
        print('master pg weights:\n', master_pgs[0][0])
        print('model pg weights:\n', model_pgs[0][0])

    history = {'train_losses': [], 'max_memory_allocation':[]}

    for e in range(2):
        model.train()
        sum_batch_losses = torch.tensor([0.], dtype=torch.float, device=gpu)
        batch_max_memory_alloc = []
        for batch_i, batch_data in enumerate(training_generator):
            if debug: 
                print(f'\nRunning batch_{batch_i}-----------------------------------------')

            batch_max_memory_alloc.append(get_max_memory_alloc())

            # 6. model forward with float16 data
            # NOTE: model zero grad, master last grad
            batch_X = batch_data['X'].cuda(gpu, non_blocking=True) # half
            batch_y = batch_data['y'].cuda(gpu, non_blocking=True) # long
            logits, activations = model(batch_X) # Half

            # 7. compute loss in float16
            # NOTE: model zero grad, master last grad
            loss = loss_criterion(logits, batch_y)  # half
            if debug:
                print('\nComputed loss')
                check_grad()
                check_weights()

            # 8. scale up loss here
            print('\nloss before scaling:', loss.item(), loss.dtype)
            loss *= loss_scale
            print('loss after scaling:', loss.item(), loss.dtype)

            # 9. backprop to compute gradients
            # NOTE: model new grad, master last grad
            # NOTE: when we call backward, gradient accumulate on model gradient, not master gradient
            loss.backward()  # half
            if debug:
                print('\nCalled backward to compute gradients')
                check_grad()
                check_weights()

            # 10. copy float16 gradients from model to float32 gradients in master copy of weights
            # NOTE: model new grad, master new grad
            to_master_grads(model_pgs, master_pgs)
            if debug:
                print('\nCopied model grad to master grad')
                check_grad()
                check_weights()

            # 11. scale down master copy gradients
            scale_down_master_grad(master_pgs, loss_scale)
            if debug:
                print('\nScaled down gradients in master copy')
                check_grad()
                check_weights()

            # 12.copy float16 gradients from model to float32 gradients in master copy of weights
            opt.step()
            if debug:
                print('\nOptimizer stepped')
                check_grad()
                check_weights()

            # 13.zero out gradients in model
            model.zero_grad()
            if debug:
                print('\nModel gradients zeroed out')
                check_grad()
                check_weights()

            # 14.copy float32 master weights to float16 model weights
            # NOTE: model zero grad, master new grad
            to_model_params(model_pgs, master_pgs)
            if debug:
                print('\nCopy master weights to model weight')
                check_grad()
                check_weights()
                            
            sum_batch_losses += loss


        num_batches = batch_i + 1.
        history['train_losses'].append(sum_batch_losses/num_batches)
        history['max_memory_allocation'] += batch_max_memory_alloc
    
    itemize = lambda x: [tensor_val.item()/loss_scale for tensor_val in x]
    history['train_losses'] = itemize(history['train_losses'])    

    return history, model

# Train

In [16]:
!nvidia-smi

Sun Nov 22 17:06:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 455.32.00    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 000047DD:00:00.0 Off |                    0 |
| N/A   46C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
with profiler.profile(profile_memory=True, record_shapes=True, use_cuda=False, with_stack=True) as prof:
    with profiler.record_function("forward"):
        history, model = main_train(args, debug=True)


model weights at init
layer 0, weight dtype torch.float32
layer 0, bias dtype torch.float32
layer 1, weight dtype torch.float32
layer 1, bias dtype torch.float32
layer 2, weight dtype torch.float32
layer 2, bias dtype torch.float32
layer 3, weight dtype torch.float32
layer 3, bias dtype torch.float32
scorer weight dtype torch.float32
scorer bias dtype torch.float32

model weights after casting
layer 0, weight dtype torch.float16
layer 0, bias dtype torch.float16
layer 1, weight dtype torch.float16
layer 1, bias dtype torch.float16
layer 2, weight dtype torch.float16
layer 2, bias dtype torch.float16
layer 3, weight dtype torch.float16
layer 3, bias dtype torch.float16
scorer weight dtype torch.float16
scorer bias dtype torch.float16

Running batch_0-----------------------------------------

Computed loss
optimizer grad:
 None
master pg grad:
 None
model pg grad:
 None
optimizer weights:
 tensor([[-0.1289,  0.0105, -0.1571,  0.1193, -0.2695,  0.2318, -0.2299, -0.2515,
         -0.1998,

/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/torch/cuda/memory.py:234: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  FutureWarning)


tensor([[-0.1289,  0.0105, -0.1571,  0.1193, -0.2695,  0.2318, -0.2299, -0.2515,
         -0.1998,  0.1432],
        [-0.1168,  0.1183, -0.2683, -0.1919, -0.1161, -0.0621, -0.2412,  0.2070,
         -0.0746,  0.1015],
        [ 0.2236,  0.0589,  0.0865,  0.3052, -0.1426,  0.1002, -0.1407,  0.2260,
          0.2524, -0.2915],
        [ 0.2700,  0.1510,  0.1378,  0.1301,  0.2629, -0.0417, -0.2673, -0.0908,
         -0.2227,  0.0209],
        [-0.0591, -0.1697, -0.0287,  0.2996, -0.0249,  0.0100, -0.0493,  0.0497,
          0.2817,  0.1934],
        [ 0.1122,  0.0687,  0.0746,  0.1222, -0.0409, -0.2939, -0.1956,  0.2700,
          0.0189, -0.2561],
        [ 0.0499,  0.2612, -0.2988, -0.2129, -0.1260,  0.0127, -0.0737, -0.0348,
         -0.3083,  0.1481],
        [ 0.2776,  0.1932, -0.2239, -0.2549,  0.1313,  0.0071,  0.1296, -0.3091,
         -0.0189,  0.2230],
        [ 0.1467,  0.0116,  0.0622, -0.0299, -0.1738, -0.1195, -0.1926,  0.2627,
          0.1741,  0.1106],
        [-0.2426,  

 tensor([[-0.2788,  0.1672,  0.0238, -0.5366,  0.0590, -0.2313,  0.2269,  0.3840,
         -0.5186,  0.4875],
        [-0.5679, -0.0563,  0.1699, -0.0338,  0.0695, -0.2069,  0.3701, -0.0789,
         -0.4741,  0.6426],
        [ 0.7925,  0.4333,  0.6313, -0.7314, -1.1006,  0.2135, -1.5195,  0.2527,
          0.7837, -0.6299],
        [-0.6255, -0.6328, -0.1174, -0.6680, -0.0582,  0.4646, -0.3479,  0.0603,
         -0.2394,  1.7266],
        [ 0.5229,  0.6533,  0.4727,  0.0843,  0.7075, -0.6187, -0.2072, -0.4336,
          0.4414,  0.1710],
        [-0.2864, -0.6733, -0.5811, -1.1514, -0.2260,  0.2661, -0.8428, -0.3711,
         -0.8711,  2.4180],
        [ 0.4114,  0.1060, -0.1840,  0.5049, -0.2231,  0.2957,  0.0325, -0.0106,
          0.2186, -0.6450],
        [-0.9302,  0.1155,  0.7334, -0.7227,  0.6450,  0.0662,  0.1432, -0.2998,
          0.6089,  0.9917],
        [-0.1735,  0.1075, -0.4229, -0.0087,  0.1104,  0.0100,  0.3557,  0.8838,
         -0.4785,  0.2861],
        [-0.1775, 

 tensor([[-0.1289,  0.0105, -0.1571,  0.1193, -0.2695,  0.2318, -0.2299, -0.2515,
         -0.1998,  0.1432],
        [-0.1168,  0.1183, -0.2683, -0.1919, -0.1161, -0.0621, -0.2412,  0.2070,
         -0.0746,  0.1015],
        [ 0.2236,  0.0589,  0.0865,  0.3052, -0.1426,  0.1002, -0.1407,  0.2260,
          0.2524, -0.2915],
        [ 0.2700,  0.1510,  0.1378,  0.1301,  0.2629, -0.0417, -0.2673, -0.0908,
         -0.2227,  0.0209],
        [-0.0591, -0.1697, -0.0287,  0.2996, -0.0249,  0.0100, -0.0493,  0.0497,
          0.2817,  0.1934],
        [ 0.1122,  0.0687,  0.0746,  0.1222, -0.0409, -0.2939, -0.1956,  0.2700,
          0.0189, -0.2561],
        [ 0.0499,  0.2612, -0.2988, -0.2129, -0.1260,  0.0127, -0.0737, -0.0348,
         -0.3083,  0.1481],
        [ 0.2776,  0.1932, -0.2239, -0.2549,  0.1313,  0.0071,  0.1296, -0.3091,
         -0.0189,  0.2230],
        [ 0.1467,  0.0116,  0.0622, -0.0299, -0.1738, -0.1195, -0.1926,  0.2627,
          0.1741,  0.1106],
        [-0.2426, 

...

 tensor([[-2.4486e-04,  5.4264e-04,  9.1851e-05, -4.6468e-04,  3.2330e-04,
          5.1355e-04,  7.7820e-04,  8.5878e-04, -9.7227e-04,  1.0071e-03],
        [-7.1001e-04,  1.0910e-03,  2.9659e-04,  2.8610e-04,  2.2316e-04,
         -5.0688e-04,  6.3801e-04, -4.2248e-04,  1.0300e-04,  9.4891e-04],
        [ 9.4652e-04, -2.5558e-04,  1.0157e-03, -2.0447e-03, -1.5774e-03,
         -2.8729e-04, -2.3460e-03,  8.8549e-04,  1.3037e-03, -2.0485e-03],
        [-7.7868e-04, -9.6130e-04,  1.0624e-03, -3.1400e-04,  1.3614e-04,
          1.4818e-04, -1.0052e-03,  2.7609e-04,  8.2302e-04,  2.8877e-03],
        [ 1.5821e-03,  1.4925e-03,  9.8896e-04, -6.9475e-04,  1.6603e-03,
         -1.6270e-03, -7.2098e-04, -1.3885e-03,  1.0881e-03, -4.1938e-04],
        [ 6.6948e-04, -9.5129e-04,  9.2760e-06, -1.7385e-03, -9.3102e-05,
         -5.4419e-05, -1.7834e-03, -7.1287e-04, -2.2812e-03,  4.5395e-03],
        [ 1.1501e-03,  2.0099e-04, -4.8518e-04,  1.0366e-03, -8.5449e-04,
         -2.2888e-04,  1.4208e-

 tensor([[-0.1289,  0.0104, -0.1572,  0.1194, -0.2696,  0.2318, -0.2299, -0.2516,
         -0.1997,  0.1431],
        [-0.1167,  0.1183, -0.2684, -0.1919, -0.1161, -0.0620, -0.2413,  0.2070,
         -0.0745,  0.1013],
        [ 0.2233,  0.0589,  0.0864,  0.3055, -0.1423,  0.1001, -0.1403,  0.2259,
          0.2522, -0.2913],
        [ 0.2702,  0.1511,  0.1378,  0.1303,  0.2630, -0.0419, -0.2672, -0.0907,
         -0.2226,  0.0205],
        [-0.0593, -0.1699, -0.0288,  0.2996, -0.0252,  0.0103, -0.0493,  0.0499,
          0.2816,  0.1934],
        [ 0.1122,  0.0690,  0.0747,  0.1225, -0.0408, -0.2941, -0.1953,  0.2701,
          0.0193, -0.2568],
        [ 0.0497,  0.2612, -0.2987, -0.2130, -0.1258,  0.0127, -0.0737, -0.0347,
         -0.3084,  0.1482],
        [ 0.2778,  0.1932, -0.2241, -0.2547,  0.1311,  0.0071,  0.1296, -0.3089,
         -0.0191,  0.2228],
        [ 0.1469,  0.0115,  0.0622, -0.0300, -0.1739, -0.1195, -0.1927,  0.2625,
          0.1742,  0.1105],
        [-0.2426, 

 Parameter containing:
tensor([[-0.1289,  0.0104, -0.1572,  0.1194, -0.2695,  0.2318, -0.2300, -0.2515,
         -0.1997,  0.1431],
        [-0.1167,  0.1183, -0.2683, -0.1919, -0.1161, -0.0620, -0.2412,  0.2070,
         -0.0745,  0.1013],
        [ 0.2234,  0.0589,  0.0864,  0.3054, -0.1422,  0.1001, -0.1404,  0.2258,
          0.2522, -0.2913],
        [ 0.2703,  0.1511,  0.1378,  0.1302,  0.2629, -0.0419, -0.2673, -0.0907,
         -0.2227,  0.0205],
        [-0.0593, -0.1699, -0.0288,  0.2996, -0.0252,  0.0103, -0.0493,  0.0499,
          0.2815,  0.1934],
        [ 0.1122,  0.0690,  0.0747,  0.1226, -0.0407, -0.2939, -0.1953,  0.2700,
          0.0193, -0.2568],
        [ 0.0497,  0.2612, -0.2988, -0.2130, -0.1259,  0.0127, -0.0737, -0.0348,
         -0.3083,  0.1482],
        [ 0.2778,  0.1932, -0.2241, -0.2546,  0.1311,  0.0071,  0.1296, -0.3088,
         -0.0191,  0.2228],
        [ 0.1470,  0.0115,  0.0623, -0.0300, -0.1738, -0.1195, -0.1927,  0.2625,
          0.1742,  0.110

 tensor([[-5.6696e-04,  6.6614e-04,  6.1417e-04, -1.0939e-03,  3.4809e-04,
         -1.5521e-04,  6.6757e-04,  2.6131e-04, -3.1114e-04,  3.6168e-04],
        [-6.5327e-04,  7.6115e-05,  2.0432e-04, -5.3167e-04,  2.7895e-05,
         -5.9319e-04,  4.7302e-04,  4.3082e-04, -3.5548e-04,  8.5688e-04],
        [ 1.9684e-03,  3.5095e-04,  1.8215e-04, -1.0862e-03, -1.8911e-03,
          2.4796e-04, -2.0065e-03, -6.5136e-04,  8.0299e-04, -1.4639e-03],
        [-1.3189e-03, -6.3086e-04,  6.6710e-04, -9.3222e-04,  7.1955e-04,
          1.7796e-03,  1.9863e-05,  5.3358e-04, -6.8307e-05,  3.0823e-03],
        [ 1.3399e-03,  1.4734e-03,  8.7118e-04,  1.4985e-04,  1.9779e-03,
         -1.1654e-03,  1.2338e-04, -1.7250e-04,  1.3857e-03, -1.6248e-04],
        [-7.9823e-04, -1.7042e-03, -1.4305e-04, -1.5774e-03, -5.8985e-04,
          6.5374e-04, -1.6928e-03, -2.9397e-04, -1.7080e-03,  3.5954e-03],
        [ 1.1339e-03, -2.8610e-04, -5.6934e-04,  9.7752e-04, -1.2293e-03,
         -2.6345e-04,  1.4758e-

 Parameter containing:
tensor([[-0.1289,  0.0104, -0.1572,  0.1194, -0.2695,  0.2318, -0.2300, -0.2515,
         -0.1997,  0.1431],
        [-0.1167,  0.1183, -0.2683, -0.1919, -0.1161, -0.0620, -0.2412,  0.2070,
         -0.0745,  0.1013],
        [ 0.2233,  0.0589,  0.0864,  0.3054, -0.1422,  0.1001, -0.1403,  0.2258,
          0.2522, -0.2913],
        [ 0.2703,  0.1511,  0.1378,  0.1302,  0.2629, -0.0419, -0.2673, -0.0907,
         -0.2227,  0.0204],
        [-0.0593, -0.1699, -0.0288,  0.2996, -0.0252,  0.0103, -0.0493,  0.0499,
          0.2815,  0.1934],
        [ 0.1122,  0.0690,  0.0747,  0.1226, -0.0407, -0.2939, -0.1953,  0.2700,
          0.0193, -0.2568],
        [ 0.0497,  0.2612, -0.2988, -0.2130, -0.1259,  0.0127, -0.0737, -0.0347,
         -0.3083,  0.1482],
        [ 0.2778,  0.1932, -0.2241, -0.2546,  0.1311,  0.0071,  0.1296, -0.3088,
         -0.0191,  0.2228],
        [ 0.1470,  0.0115,  0.0623, -0.0300, -0.1738, -0.1195, -0.1927,  0.2625,
          0.1742,  0.110

# Training Results

In [27]:
# Ran without profiler
history

{'train_losses': [1.61328125,
  1.6127387285232544,
  1.6127387285232544,
  1.6124131679534912,
  1.6119791269302368,
  1.6117621660232544,
  1.611328125,
  1.6112196445465088,
  1.6108940839767456,
  1.6108940839767456,
  1.610568642616272,
  1.6101346015930176,
  1.6101346015930176,
  1.6098090410232544,
  1.6097005605697632,
  1.6094834804534912,
  1.6092665195465088,
  1.6090494394302368,
  1.6089409589767456,
  1.608615517616272],
 'max_memory_allocation': [{device(type='cuda', index=0): 0.010752},
  {device(type='cuda', index=0): 0.256512},
  {device(type='cuda', index=0): 0.266752},
  {device(type='cuda', index=0): 0.266752},
  {device(type='cuda', index=0): 0.266752},
  {device(type='cuda', index=0): 0.266752},
  {device(type='cuda', index=0): 0.266752},
  {device(type='cuda', index=0): 0.266752},
  {device(type='cuda', index=0): 0.266752},
  {device(type='cuda', index=0): 0.266752},
  {device(type='cuda', index=0): 0.267264},
  {device(type='cuda', index=0): 0.267264},
  {devi

# Profiler Results

In [19]:
rankByCriteria(prof, model, criteria='cuda_memory_usage', per_thread=False, per_inp_shapes=False, include_external=False)

Ranked by cuda_memory_usage

41.18 Mb
##############################################
model, aten::empty, forward, (26) last_X = torch.relu(last_X)
2.81 Mb
##############################################
model, aten::addmm, forward, (24) last_X = linear_layer(last_X)
1.41 Mb
##############################################
model.scorer, aten::addmm, forward, (93) return F.linear(input, self.weight, self.bias)
model, aten::addmm, forward, (24) last_X = linear_layer(last_X)
1.41 Mb
##############################################
model, aten::resize_, forward, (24) last_X = linear_layer(last_X)
1.41 Mb
##############################################
model.scorer, aten::resize_, forward, (93) return F.linear(input, self.weight, self.bias)
model, aten::resize_, forward, (24) last_X = linear_layer(last_X)
1.41 Mb
##############################################
model, aten::relu, forward, (26) last_X = torch.relu(last_X)
1.41 Mb
##############################################
model, aten::threshold, 

# Nvidia Results

In [28]:
# Ran without profiler
!nvidia-smi

Sun Nov 22 16:36:20 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 455.32.00    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 000047DD:00:00.0 Off |                    0 |
| N/A   43C    P0    55W / 149W |    338MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [29]:
# Ran without profiler
print(torch.cuda.memory_summary(cuda0))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |  276480 B  |  160123 KB |  160123 KB |
|       from large pool |       0 B  |       0 B  |       0 KB |       0 KB |
|       from small pool |       0 B  |  276480 B  |  160123 KB |  160123 KB |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |  276480 B  |  160123 KB |  160123 KB |
|       from large pool |       0 B  |       0 B  |       0 KB |